In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/mix-unmix-master

/content/drive/MyDrive/mix-unmix-master


In [3]:
%pwd

'/content/drive/MyDrive/mix-unmix-master'

In [4]:
# !unrar x data_full_size.rar

In [5]:
# import os.path
# os.path.isfile('data_full_size/coco/train/2786_14_jpg.rf.2b71663c319101fba3adc94035c108d1.jpg')

In [6]:
# !unrar x 'data_full_size.rar'

In [7]:
# # get the Detectron2 v0.5 package
# !wget https://github.com/facebookresearch/detectron2/archive/refs/tags/v0.5.zip

# # unzip
# !unzip v0.5.zip

# install
!python -m pip install -e detectron2-0.5


Obtaining file:///content/drive/MyDrive/mix-unmix-master/detectron2-0.5
  Preparing metadata (setup.py) ... done
  Attempting uninstall: detectron2
    Found existing installation: detectron2 0.5
    Uninstalling detectron2-0.5:
      Successfully uninstalled detectron2-0.5
  Running setup.py develop for detectron2


In [8]:
!pip install timm

##Train

In [ ]:
!python train_net.py --num-gpus 1 --config configs/mum_configs/coco_custom.yaml

Command Line Args: Namespace(config_file='configs/mum_configs/coco_custom.yaml', resume=False, eval_only=False, num_gpus=1, num_machines=1, machine_rank=0, dist_url='tcp://127.0.0.1:49152', opts=[])
[11/26 16:25:14 detectron2]: Rank of current process: 0. World size: 1
[11/26 16:25:15 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------------
sys.platform            linux
Python                  3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
numpy                   1.23.5
detectron2              0.5 @/content/drive/MyDrive/mix-unmix-master/detectron2-0.5/detectron2
Compiler                GCC 11.4
CUDA compiler           not available
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 2.1.0+cu118 @/usr/local/lib/python3.10/dist-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   Tesla T4 (arch=7.5)
CUDA_HOME               /usr/local/cuda
Pillow                  9.4

## Inference

In [ ]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("TRAIN_DATASET", {}, "data_full_size/coco/annotations/instances_train2017.json", "data_full_size/coco/train")
register_coco_instances("VAL_DATASET", {}, "data_full_size/coco/annotations/instances_val2017.json", "data_full_size/coco/valid")

In [ ]:
import os
from detectron2.config import get_cfg

cfg = get_cfg()

cfg.merge_from_file("configs/Base-RCNN-FPN.yaml")
cfg.MODEL.WEIGHTS = os.path.join("logs/coco/model_final.pth")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50   # Testing threshold
cfg.DATASETS.TRAIN = ("TRAIN_DATASET", )
cfg.DATASETS.TEST = ("VAL_DATASET", )


In [ ]:
from detectron2.data import DatasetCatalog
import cv2, random
from google.colab.patches import cv2_imshow
from detectron2.utils.visualizer import Visualizer
from mum.engine.trainer import CustomPredictor

dataset_dicts = DatasetCatalog.get("VAL_DATASET")
predictor = CustomPredictor(cfg)

for d in random.sample(dataset_dicts, 2):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   scale=1
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])
    #
    annotations = d.get("annotations", None)
    if annotations is not None:
        for annotation in annotations:
            label = annotation["category_id"]
            bbox = annotation["bbox"]
            x, y, w, h = [int(coord) for coord in bbox]
            cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 2)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            font_color = (255, 255, 255)
            cv2.putText(im, str(label), (x, y - 10), font, font_scale, font_color, 2)

        # Hiển thị hình ảnh gốc với bounding boxes và nhãn gốc
        cv2_imshow(im)